In [1]:
# E-power configuration (3 phase)

import numpy as np
from itertools import combinations
import math
import random


# Sample data
ui_elements = [[[41, 328, 309, 348], "B1"], [[40, 348, 309, 369], "B2"], [[42, 379, 282, 397], "B3"], [[37, 403, 274, 426], "B4"], [[37, 427, 277, 455], "B5"], [[38, 453, 277, 473], "B6"], [[42, 477, 319, 498], "B7"], [[35, 501, 287, 526], "B8"], [[361, 322, 670, 350], "B9"], [[364, 350, 659, 375], "B10"], [[368, 375, 669, 409], "B11"], [[367, 519, 673, 551], "B12"], [[365, 548, 682, 577], "B13"], [[743, 321, 1014, 353], "B14"], [[737, 349, 1064, 380], "B15"], [[735, 380, 1046, 404], "B16"], [[739, 402, 1112, 425], "B17"], [[735, 425, 1105, 447], "B18"], [[732, 451, 1096, 478], "B19"], [[736, 474, 1117, 512], "B20"]]

association_data = [
    ('B1','B2','B3','B4','B5','B6','B7','B8'),
    ('B9','B10','B11'),
    ('B12','B13'),    
    ('B14','B15','B16','B17','B18','B19','B20')
]


trial_history = data = [
    [1, 0, "B1"], [1, 2.99187, "B2"], [1, 4.367566, "B4"], [1, 2.560202, "B5"], [1, 5.176222, "B7"], 
    [1, 2.175851, "B7"], [1, 3.783863, "B1"], [1, 5.200065, "B2"], [1, 7.511943, "B6"], [1, 2.312128, "B9"], 
    [1, 1.007833, "B9"], [1, 1.864001, "B10"], [1, 0.960009, "B10"], [1, 2.080011, "B11"], [1, 0.807891, "B11"], 
    [1, 2.415482, "B12"], [1, 0.888548, "B12"], [1, 1.271421, "B13"], [1, 0.552566, "B13"], [1, 0.65342, "B16"], 
    [1, 3.024015, "B18"], [1, 2.447566, "B17"], [2, 0, "B1"], [2, 2.024074, "B2"], [2, 1.719922, "B2"], 
    [2, 3.488064, "B3"], [2, 0.584041, "B3"], [2, 1.57618, "B4"], [2, 0.839705, "B5"], [2, 1.879955, "B6"], 
    [2, 3.192052, "B7"], [2, 1.407767, "B7"], [2, 3.584327, "B8"], [2, 1.567627, "B9"], [2, 1.984079, "B9"], 
    [2, 2.784193, "B10"], [2, 0.28745, "B10"], [2, 2.240335, "B11"], [2, 0.224075, "B11"], [2, 4.120113, "B13"], 
    [2, 1.255841, "B13"], [2, 4.687919, "B16"], [2, 2.687865, "B17"], [2, 1.575998, "B19"], [3, 0, "B1"], 
    [3, 1.063996, "B2"], [3, 2.839851, "B3"], [3, 5.648232, "B7"], [3, 1.055837, "B7"], [3, 6.376153, "B6"], 
    [3, 4.511765, "B5"], [3, 0.919991, "B4"], [3, 1.72004, "B9"], [3, 2.007884, "B9"], [3, 1.431977, "B10"], 
    [3, 1.920039, "B10"], [3, 1.608142, "B11"], [3, 0.879885, "B11"], [3, 2.271982, "B12"], [3, 0.479367, "B13"], 
    [3, 5.624588, "B17"], [3, 1.375425, "B18"], [3, 3.608582, "B19"], [4, 7.239898, "B1"], [4, 1.696, "B1"], 
    [4, 1, "B2"], [4, 1.048016, "B2"], [4, 1.391863, "B3"], [4, 0.879909, "B3"], [4, 4.287979, "B7"], 
    [4, 0.712059, "B7"], [4, 0.999636, "B6"], [4, 2.039745, "B5"], [4, 1.99262, "B9"], [4, 0.488186, "B9"], 
    [4, 0.687919, "B10"], [4, 0.567891, "B10"], [4, 3.560232, "B11"], [4, 0.407745, "B11"], [4, 1.783922, "B12"], 
    [2, 2.457854, "B16"], [4, 1.63224, "B14"], [4, 13.359923, "B17"], [4, 0.648188, "B18"], [4, 4.647777, "B19"], 
    [5, 0, "B1"], [5, 0.82416, "B1"], [5, 2.799267, "B2"], [5, 0.792538, "B2"], [5, 1.568251, "B3"], 
    [5, 0.511671, "B3"], [5, 2.191639, "B5"], [5, 1.424469, "B5"], [5, 1.199949, "B4"], [5, 1.183981, "B3"], 
    [5, 1.264107, "B4"], [5, 2.175287, "B3"], [5, 1.760209, "B8"], [5, 1.312291, "B4"], [5, 0.655937, "B4"], 
    [5, 3.103824, "B11"], [5, 0.871738, "B11"], [5, 1.000039, "B9"], [5, 6.223, "B15"], [5, 1.252, "B16"], 
    [5, 2.832181, "B17"], [5, 0.496269, "B18"], [5, 0.592229, "B19"], [5, 1.727656, "B17"], [5, 0.960024, "B17"], 
    [5, 0.820067, "B20"]
]

In [2]:
def create_ui_grid(ui_elements, num_rows, num_cols, area_width=2000, area_height=2000):
    grid = np.zeros((num_rows, num_cols), dtype=object)

    cell_width = area_width / num_cols
    cell_height = area_height / num_rows

    for element in ui_elements:
        coordinates, name = element
        x1, y1, x2, y2 = coordinates
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2

        row = int(center_y // cell_height)
        col = int(center_x // cell_width)

        grid[row, col] = name

    for i in range(num_rows):
        for j in range(num_cols):
            if grid[i, j] == 0:
                grid[i, j] = 0

    return grid

In [3]:
#epower 3phase
num_rows = 80
num_cols = 5


ui_elements_grid = create_ui_grid(ui_elements, num_rows, num_cols)

In [4]:
ui_elements_grid

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ['B1', 'B9', 'B14', 0, 0],
       ['B2', 'B10', 'B15', 0, 0],
       ['B3', 'B11', 'B16', 0, 0],
       ['B4', 0, 'B17', 0, 0],
       ['B5', 0, 'B18', 0, 0],
       ['B6', 0, 'B19', 0, 0],
       ['B7', 0, 'B20', 0, 0],
       ['B8', 0, 0, 0, 0],
       [0, 'B12', 0, 0, 0],
       [0, 'B13', 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 

In [5]:
# This function creates cordinates within the ui_grid so that we can get adaptation of elemnts moving within their associations

def compute_smallest_bounding_box_matrix(grid, association_data):
    bounding_boxes = {}
    for group in association_data:
        min_row = min_col = float('inf')
        max_row = max_col = float('-inf')

        for element in group:
            for row in range(grid.shape[0]):
                for col in range(grid.shape[1]):
                    if grid[row, col] == element:
                        if row < min_row:
                            min_row = row
                        if col < min_col:
                            min_col = col
                        if row > max_row:
                            max_row = row
                        if col > max_col:
                            max_col = col

        bounding_box = [(min_row, min_col), (max_row, max_col)]
        for element in group:
            bounding_boxes[element] = bounding_box

    return bounding_boxes

In [6]:
class Utilities:
    def __init__(self, ui_elements_grid, association_data):
        self.ui_elements_grid = ui_elements_grid
        self.associations = self.parse_associations(association_data)
    
    def parse_associations(self, association_data):
        associations = {}
        for group in association_data:
            for element in group:
                associations[element] = group
        return associations
    
    def get_click_distribution(self, history, normalize=True):
        frequency = {}
        total_clicks = 0
        indexed_history = []

        # Initialize frequency dictionary
        for row in self.ui_elements_grid:
            for command in row:
                if command != 0:
                    frequency[command] = 0

        # Calculate frequency of each command
        for trial in history:
            _, _, command = trial
            if command != 0:
                if command in frequency:
                    frequency[command] += 1
                else:
                    frequency[command] = 1
                total_clicks += 1

        # Normalize frequencies if required
        if normalize:
            for command in frequency.keys():
                frequency[command] = round(frequency[command] / total_clicks, 3)

        # Create indexed history
        for trial in history:
            _, _, command = trial
            if command != 0:
                command_position = np.argwhere(self.ui_elements_grid == command)
                if command_position.size > 0:
                    indexed_history.append([command, command_position[0].tolist()])

        return frequency, total_clicks, indexed_history


    
    def get_activations(self, indexed_history, session_interval, duration_between_clicks):
        activations = {} # Activation per target per location
        session_click_length = 20 # Clicks per session
        total_clicks = len(indexed_history) # Total clicks from indexed history
        total_sessions = math.ceil(total_clicks / session_click_length) # Number of sessions so far

        for i in range(total_clicks):
            session = math.ceil((i + 1) / session_click_length) # Session index
            item = indexed_history[i][0]
            position = tuple(indexed_history[i][1]) # Position from indexed history

            if item not in activations:
                activations[item] = {position: 0} # Item has not been seen yet. Add to dictionary
            if position not in activations[item]:
                activations[item][position] = 0 # Item not seen at this position yet. Add to item's dictionary

            time_difference = duration_between_clicks * (total_clicks - i) + (total_sessions - session) * session_interval # Difference between time now and time of click
            activations[item][position] += pow(time_difference, -0.5)

        # Flatten activations to get mean value and element position
        activations = {k: (np.mean(list(v.values())), list(v.keys())[0]) for k, v in activations.items()}
        return activations
    
    
    def get_activations_initial(self, indexed_history, session_interval, duration_between_clicks):
        activations = {} # Activation per target per location
        session_click_length = 20 # Clicks per session
        total_clicks = len(indexed_history) # Total clicks from indexed history
        total_sessions = math.ceil(total_clicks / session_click_length) # Number of sessions so far
    
        # Ensure the length of duration_between_clicks matches the number of clicks
        if len(duration_between_clicks) != total_clicks:
            raise ValueError("Length of duration_between_clicks must match the number of clicks in indexed_history.")
    
        for i in range(total_clicks):
            session = math.ceil((i + 1) / session_click_length) # Session index
            item = indexed_history[i][0]
            position = tuple(indexed_history[i][1]) # Position from indexed history
    
            if item not in activations:
                activations[item] = {position: 0} # Item has not been seen yet. Add to dictionary
            if position not in activations[item]:
                activations[item][position] = 0 # Item not seen at this position yet. Add to item's dictionary
    
            time_difference = duration_between_clicks[i] + (total_sessions - session) * session_interval # Difference between time now and time of click
            activations[item][position] += pow(time_difference, -0.5)
    
        # Flatten activations to get mean value and element position
        activations = {k: (np.mean(list(v.values())), list(v.keys())[0]) for k, v in activations.items()}
        return activations

    
    def get_association_matrix(self):
        rows, cols = self.ui_elements_grid.shape
        association_matrix = np.zeros((rows * cols, rows * cols))
        
        for i in range(rows):
            for j in range(cols):
                if self.ui_elements_grid[i, j] in self.associations:
                    for k in range(rows):
                        for l in range(cols):
                            if self.ui_elements_grid[k, l] in self.associations[self.ui_elements_grid[i, j]]:
                                association_matrix[i * cols + j, k * cols + l] = 1.0
                            else:
                                association_matrix[i * cols + j, k * cols + l] = 0.0
                else:
                    for k in range(rows):
                        for l in range(cols):
                            association_matrix[i * cols + j, k * cols + l] = 0.0
        
        return association_matrix

    def get_sorted_frequencies(self, frequency):
        sorted_frequencies = []
        
        for row in self.ui_elements_grid:
            for item in row:
                if item == 0:
                    sorted_frequencies.append(0.0)
                else:
                    sorted_frequencies.append(frequency.get(item, 0.0))
                    
        return sorted_frequencies



    def update_freqdist(self, history, grid, normalize=True):
        freqdist = {}
        for row in grid:
            for command in row:
                if command != 0:
                    freqdist[command] = 0
    
        simple_hist = [row[0] for row in history]
        for item in simple_hist:
            if item == "":
                continue
            if item not in list(freqdist.keys()):
                freqdist[item] = 1.
            else:
                freqdist[item] += 1.
    
        total_clicks = sum(list(freqdist.values()))
    
        if normalize:
            for command in list(freqdist.keys()):
                freqdist[command] = round(freqdist[command] / total_clicks, 3)
    
        return freqdist
    
    
    
    
    def update_hist(self, indexed_history, new_grid, number_of_clicks=20):
        history = indexed_history.copy()
        #print(len(history))
            
        if number_of_clicks:
            clicks_to_add = history[-number_of_clicks:]
        
            
            for item, position in clicks_to_add:
                #history.append([item, new_grid_list.index(click)])
                #print(new_grid_list.index(item))
                result = np.where(new_grid == item)
                result_list = list(zip(result[0], result[1]))
                history.append([item, result_list])
    
        freqdist=update_freqdist(history, new_grid)
        return freqdist, history

In [7]:
# mcts stuff

class MenuState:
    def __init__(self, grid, associations):
        self.grid = grid
        self.associations = associations
        self.num_rows, self.num_cols = grid.shape
        self.element_positions = {grid[r, c]: (r, c) for r in range(self.num_rows) for c in range(self.num_cols) if grid[r, c] != 0}
        self.update_bounding_boxes()

    def update_bounding_boxes(self):
        self.bounding_boxes = compute_smallest_bounding_box_matrix(self.grid, list(self.associations.values()))

    def update_element_position(self):
        self.element_positions = {self.grid[r, c]: (r, c) for r in range(self.num_rows) for c in range(self.num_cols) if self.grid[r, c] != 0}

    def move_element(self, element, new_pos):
        ref_pos = self.element_positions[element]
        new_row, new_col = new_pos

        if self.grid[new_row, new_col] == 0 and 0 <= new_row < self.num_rows and 0 <= new_col < self.num_cols:
            self.grid[ref_pos[0], ref_pos[1]] = 0
            self.grid[new_row, new_col] = element
            self.element_positions[element] = new_pos

    def move_associations_mcts(self, moves):
        for move in moves:
            element = move['element']
            new_pos = move['new_position']
            self.move_element(element, new_pos)
        
        self.update_bounding_boxes()
        self.update_element_position()
        return self.grid

    def get_move_association_adaptations(self):
        possible_moves = []
        for element, pos in self.element_positions.items():
            if element in self.associations:
                associated_elements = self.associations[element]
                ref_row, ref_col = pos

                for new_row in range(self.num_rows):
                    for new_col in range(self.num_cols):
                        if self.grid[new_row, new_col] != 0:
                            continue  # Skip if the position is not empty

                        valid_move = True
                        new_positions = []
                        for assoc_element in associated_elements:
                            cur_pos = self.element_positions[assoc_element]
                            cur_row, cur_col = cur_pos
                            rel_row, rel_col = cur_row - ref_row, cur_col - ref_col
                            new_assoc_row, new_assoc_col = new_row + rel_row, new_col + rel_col

                            if not (0 <= new_assoc_row < self.num_rows and 0 <= new_assoc_col < self.num_cols):
                                valid_move = False
                                break

                            if self.grid[new_assoc_row, new_assoc_col] != 0:
                                valid_move = False
                                break

                            new_positions.append({
                                'element': assoc_element,
                                'current_position': cur_pos,
                                'new_position': (new_assoc_row, new_assoc_col),
                                'type': 'move'
                            })

                        if valid_move:
                            possible_moves.append(new_positions)
        return possible_moves

    def get_swap_association_adaptations(self):
        possible_moves = []
        unique_associations = {tuple(v) for v in self.associations.values()}
        unique_associations = list(unique_associations)
    
        for i in range(len(unique_associations)):
            for j in range(len(unique_associations)):
                if i >= j:
                    continue  # Skip the same association and already processed pairs
    
                assoc1 = unique_associations[i]
                assoc2 = unique_associations[j]
    
                pos1 = min(self.element_positions[elem] for elem in assoc1)
                pos2 = min(self.element_positions[elem] for elem in assoc2)
    
                rel_positions1 = {elem: (self.element_positions[elem][0] - pos1[0], self.element_positions[elem][1] - pos1[1]) for elem in assoc1}
                rel_positions2 = {elem: (self.element_positions[elem][0] - pos2[0], self.element_positions[elem][1] - pos2[1]) for elem in assoc2}
    
                swap_positions = []
                for elem in assoc1:
                    rel_pos = rel_positions1[elem]
                    new_pos = (pos2[0] + rel_pos[0], pos2[1] + rel_pos[1])
                    swap_positions.append({
                        'element': elem,
                        'current_position': self.element_positions[elem],
                        'new_position': new_pos
                    })
    
                for elem in assoc2:
                    rel_pos = rel_positions2[elem]
                    new_pos = (pos1[0] + rel_pos[0], pos1[1] + rel_pos[1])
                    swap_positions.append({
                        'element': elem,
                        'current_position': self.element_positions[elem],
                        'new_position': new_pos
                    })
    
                possible_moves.append(swap_positions)
        return possible_moves

    def apply_association_swap_adaptation(self, adaptation):
        current_positions = {}
        new_positions = {}

        for move in adaptation:
            element = move['element']
            current_pos = self.element_positions[element]
            new_pos = move['new_position']
            current_positions[element] = current_pos
            new_positions[element] = new_pos

        for element, pos in current_positions.items():
            self.grid[pos[0], pos[1]] = 0

        for element, new_pos in new_positions.items():
            self.grid[new_pos[0], new_pos[1]] = element
            self.element_positions[element] = new_pos

        self.update_bounding_boxes()
        self.update_element_position()
        return self.grid

# Assuming that compute_smallest_bounding_box_matrix is defined elsewhere


In [8]:
utilities = Utilities(ui_elements_grid, association_data)

In [9]:
# Create the UserState object
menustate = MenuState(ui_elements_grid, utilities.associations)

In [10]:
# MCTS stuff

class Adaptations:
    def __init__(self, menustate):
        self.menustate = menustate

    def get_move_adaptations_within_boundingbox(self, grid, element_positions):
        bounding_boxes = compute_smallest_bounding_box_matrix(grid, list(self.menustate.associations.values()))
        possible_adaptations = []

        for element, assoc_position in bounding_boxes.items():
            min_row, min_col = assoc_position[0]
            max_row, max_col = assoc_position[1]

            for assoc_element in self.menustate.associations[element]:
                for row in range(min_row, max_row + 1):
                    for col in range(min_col, max_col + 1):
                        if grid[row, col] == 0:
                            new_pos = (row, col)
                            possible_adaptations.append({
                                'element': assoc_element,
                                'current_position': element_positions[assoc_element],
                                'new_position': new_pos
                            })

        return possible_adaptations

    def get_all_adaptations_within_bounding_box_mcts(self):
        self.menustate.update_bounding_boxes()
        self.menustate.update_element_position()
        possible_adaptations = []
        base_adaptations = self.menustate.get_move_association_adaptations()

        for adaptation in base_adaptations:
            temp_grid = self.menustate.grid.copy()
            temp_element_positions = self.menustate.element_positions.copy()
            self.menustate.move_associations_mcts(adaptation)

            bounding_box_adaptations = self.get_move_adaptations_within_boundingbox(self.menustate.grid, temp_element_positions)

            if not bounding_box_adaptations:
                possible_adaptations.append({
                    'base_adaptation': adaptation,
                    'bbox_adaptation': [None]
                })
            else:
                for bbox_adaptation in bounding_box_adaptations:
                    possible_adaptations.append({
                        'base_adaptation': adaptation,
                        'bbox_adaptation': [bbox_adaptation]
                    })

            self.menustate.grid = temp_grid
            self.menustate.element_positions = temp_element_positions

        return possible_adaptations

    def move_elements_mcts(self, adaptation):
        self.menustate.update_bounding_boxes()
        self.menustate.update_element_position()

        for move in adaptation['base_adaptation']:
            element = move['element']
            new_pos = move['new_position']
            self.menustate.move_element(element, new_pos)

        if adaptation['bbox_adaptation'][0] is not None:
            for move in adaptation['bbox_adaptation']:
                element = move['element']
                new_pos = move['new_position']
                self.menustate.move_element(element, new_pos)
        
        return self.menustate.grid

    def get_swap_adaptations_within_boundingbox(self, grid, element_positions):
        bounding_boxes = compute_smallest_bounding_box_matrix(grid, list(self.menustate.associations.values()))
        possible_adaptations = []

        for element, assoc_position in bounding_boxes.items():
            min_row, min_col = assoc_position[0]
            max_row, max_col = assoc_position[1]

            for assoc_element in self.menustate.associations[element]:
                for row in range(min_row, max_row + 1):
                    for col in range(min_col, max_col + 1):
                        if grid[row, col] != 0 and (row, col) != element_positions[assoc_element]:
                            new_pos = (row, col)
                            current_element_at_new_pos = grid[row, col]
                            possible_adaptations.append([
                                {
                                    'element': assoc_element,
                                    'current_position': element_positions[assoc_element],
                                    'new_position': new_pos
                                },
                                {
                                    'element': current_element_at_new_pos,
                                    'current_position': new_pos,
                                    'new_position': element_positions[assoc_element]
                                }
                            ])

        return possible_adaptations

    def get_all_swap_adaptations_within_bounding_box_mcts(self):
        self.menustate.update_bounding_boxes()
        self.menustate.update_element_position()
        possible_adaptations = []
        swap_adaptations = self.menustate.get_swap_association_adaptations()
    
        for swap_adaptation in swap_adaptations:
            temp_grid = self.menustate.grid.copy()
            temp_element_positions = self.menustate.element_positions.copy()
    
            self.menustate.apply_association_swap_adaptation(swap_adaptation)
    
            self.menustate.update_bounding_boxes()
            self.menustate.update_element_position()
    
            temp_element_positions2 = self.menustate.element_positions.copy()
    
            try:
                bounding_box_adaptations = self.get_swap_adaptations_within_boundingbox(self.menustate.grid, temp_element_positions2)
    
                if not bounding_box_adaptations:
                    possible_adaptations.append({
                        'base_adaptation': swap_adaptation,
                        'bbox_adaptation': [None]
                    })
                else:
                    for bbox_adaptation in bounding_box_adaptations:
                        possible_adaptations.append({
                            'base_adaptation': swap_adaptation,
                            'bbox_adaptation': [bbox_adaptation]
                        })
            except Exception as e:
                print(f"Skipping swap adaptation due to error: {e}")
    
            self.menustate.grid = temp_grid
            self.menustate.element_positions = temp_element_positions
    
        return possible_adaptations
    
    def apply_bbox_swap_adaptation_mcts(self, adaptation):
        temp_grid = self.menustate.grid.copy()
        temp_element_positions = self.menustate.element_positions.copy()
        
        self.menustate.apply_association_swap_adaptation(adaptation['base_adaptation'])
    
        if adaptation['bbox_adaptation'][0] is not None:
            temp_positions = {}
            for move_pair in adaptation['bbox_adaptation']:
                for move in move_pair:
                    element = move['element']
                    new_pos = move['new_position']
                    temp_positions[element] = new_pos
            
            for move_pair in adaptation['bbox_adaptation']:
                for move in move_pair:
                    element = move['element']
                    old_pos = self.menustate.element_positions[element]
                    self.menustate.grid[old_pos] = 0
    
            for element, new_pos in temp_positions.items():
                self.menustate.grid[new_pos] = element
                self.menustate.element_positions[element] = new_pos
        
        result_grid = self.menustate.grid
        self.menustate.grid = temp_grid
        self.menustate.element_positions = temp_element_positions
    
        self.menustate.update_bounding_boxes()
        self.menustate.update_element_position()
        return result_grid


In [11]:
class UserOracle:
    def __init__(self, maxdepth, associations, activations):
        self.maxdepth = maxdepth
        self.alpha = 2.0
        self.groupreadingcost = 0.5
        self.vicinity = 1
        self.surprisecost = 0.2
        self.point_const = 0.4
        self.associations = associations
        self.activations = activations

    def read(self, item, novice=False):
        if novice:
            return self.alpha
        if item not in self.activations:
            return self.alpha
        total_activation, _ = self.activations[item]
        return self.alpha / (1 + total_activation)
    
    def serialsearch(self, target, current_grid, previous_grid=None, novice=False):
        if previous_grid is None:
            previous_grid = current_grid
        t = 0.0
        if target not in self.activations:
            return 0.0
        
        _, target_pos = self.activations[target]
        targetlocation = target_pos
        expectedlocation = np.argwhere(previous_grid == target)[0]

        if targetlocation[0] < expectedlocation[0] or (targetlocation[0] == expectedlocation[0] and targetlocation[1] <= expectedlocation[1]):
            # Target appears at, or before, previously seen location. Read serially till found
            for i in range(targetlocation[0] + 1):
                for j in range(targetlocation[1] + 1):
                    if current_grid[i, j] != 0:
                        t += self.read(current_grid[i, j], novice)
        else:
            # Target position adapted => moved to a position after expected.
            # First, read at regular speed till expected position
            for i in range(expectedlocation[0] + 1):
                for j in range(expectedlocation[1] + 1):
                    if current_grid[i, j] != 0:
                        t += self.read(current_grid[i, j], novice)
            # Target not found yet. 
            t += self.surprisecost
            for i in range(expectedlocation[0], targetlocation[0] + 1):
                for j in range(expectedlocation[1], targetlocation[1] + 1):
                    if current_grid[i, j] != 0:
                        t += self.read(current_grid[i, j], novice=True)
        return round(t, 5)
    
    def forage(self, target, current_grid, previous_grid=None):
        if previous_grid is None:
            previous_grid = current_grid
        t = 0.0
        if target not in self.activations:
            return 0.0
        
        associated_elements = self.associations[target]
        for assoc_element in associated_elements:
            _, assoc_pos = self.activations[assoc_element]
            t += self.read(assoc_element)
            if assoc_element == target:
                return round(t, 5)
            
            start_pos = assoc_pos
            for i in range(start_pos[0], current_grid.shape[0]):
                for j in range(start_pos[1], current_grid.shape[1]):
                    if current_grid[i, j] == target:
                        t += self.read(target)
                        return round(t, 5)
                    elif current_grid[i, j] != 0 and current_grid[i, j] not in associated_elements:
                        t += self.surprisecost
                t += self.groupreadingcost
        return round(t, 5)

    def recall(self, target, current_grid, previous_grid=None):
        if previous_grid is None:
            previous_grid = current_grid
        if target not in self.activations:
            return self.serialsearch(target, current_grid, previous_grid)
        
        max_activation, _ = self.activations[target]
        if max_activation < 0.5:
            return self.surprisecost + self.serialsearch(target, current_grid, previous_grid)
        else:
            t = 0.0
            for element, (activation, position) in sorted(self.activations.items(), key=lambda x: x[1][0], reverse=True):
                t += self.read(element)
                if element == target:
                    return round(t, 5)
            return round(t + self.serialsearch(target, current_grid, previous_grid), 5)
    
    def get_average_times(self, frequency, current_grid, previous_grid=None):
        if previous_grid is None:
            previous_grid = current_grid
        serial_time = 0.0
        forage_time = 0.0
        recall_time = 0.0
        for row in range(current_grid.shape[0]):
            for col in range(current_grid.shape[1]):
                item = current_grid[row, col]
                if item == 0:
                    continue
                serial_time += frequency.get(item, 0) * self.serialsearch(item, current_grid, previous_grid)
                forage_time += frequency.get(item, 0) * self.forage(item, current_grid, previous_grid)
                recall_time += frequency.get(item, 0) * self.recall(item, current_grid, previous_grid)
        return serial_time, forage_time, recall_time
    
    def get_individual_rewards(self, current_grid, previous_grid, freqdist):
        current_grid = current_grid
        previous_grid = previous_grid if previous_grid is not None else current_grid
        frequency = freqdist
        
        new_serial_time, new_forage_time, new_recall_time = self.get_average_times(frequency, current_grid, previous_grid)
        
        if previous_grid is None:
            return [0.0, 0.0, 0.0], [new_serial_time, new_forage_time, new_recall_time]
        
        previous_serial_time, previous_forage_time, previous_recall_time = self.get_average_times(frequency, previous_grid)
        
        reward_serial = previous_serial_time - new_serial_time
        reward_forage = previous_forage_time - new_forage_time
        reward_recall = previous_recall_time - new_recall_time
        
        return [reward_serial, reward_forage, reward_recall], [new_serial_time, new_forage_time, new_recall_time]
    
    def is_terminal(self, depth=20):
        return depth >= self.maxdepth
    
    def __str__(self):
        return str(self.maxdepth)

In [12]:
def extract_times(trial_history):
    time_duration_list = [entry[1] for entry in trial_history]
    return time_duration_list

# Extract the list of times (middle values) from trial_history
time_duration_list = extract_times(trial_history)


In [13]:
# Create the Adaptations object
adaptations = Adaptations(menustate)

In [14]:
frequency, total_clicks, indexed_history = utilities.get_click_distribution(trial_history,normalize=True)
activations = utilities.get_activations_initial(indexed_history, session_interval=50, duration_between_clicks=time_duration_list)
association_matrix = utilities.get_association_matrix()

In [15]:
from copy import deepcopy

class State:
    # Initialize the state
    def __init__(self, indexed_history, freqdist, ui_elements_grid, association_matrix, previous_seen_state=None, depth=0, exposed=False):
        self.indexed_history = indexed_history
        self.association_matrix = association_matrix
        self.previous_seen_state = previous_seen_state
        self.ui_elements_grid_state = ui_elements_grid.copy()
        self.depth = depth
        self.exposed = exposed
        self.freqdist = freqdist

    # Function called when an adaptation is made. The user state and menu state are updated accordingly
    def take_adaptation(self, adaptation, update_user=True):
        new_state = deepcopy(self)
        new_state.depth += 1
        new_state.exposed = adaptation.get('expose', False)
        if self.exposed:
            new_state.previous_seen_state = self

        # Simulate the next user session by adding clicks
        if self.exposed and update_user:
            new_state.user_state.update(menu=self.user_state.grid, number_of_clicks=self.user_state.num_rows * self.user_state.num_cols)

        # Apply the adaptation
        new_state.user_state.grid = self.utilities.apply_adaptation(self.user_state.grid, adaptation)

        return new_state

In [16]:
root_state = State(indexed_history, frequency, ui_elements_grid, association_matrix, exposed=True)

In [17]:
#adaptations.get_all_adaptations_within_bounding_box()

In [18]:
def update_freqdist(history, grid, normalize=True):
    freqdist = {}
    for row in grid:
        for command in row:
            if command != 0:
                freqdist[command] = 0

    simple_hist = [row[0] for row in history]
    for item in simple_hist:
        if item == "":
            continue
        if item not in list(freqdist.keys()):
            freqdist[item] = 1.
        else:
            freqdist[item] += 1.

    total_clicks = sum(list(freqdist.values()))

    if normalize:
        for command in list(freqdist.keys()):
            freqdist[command] = round(freqdist[command] / total_clicks, 3)

    return freqdist




def update_hist(indexed_history, new_grid, number_of_clicks=20):
    history = indexed_history.copy()
    #print(len(history))
        
    if number_of_clicks:
        clicks_to_add = history[-number_of_clicks:]
    
        
        for item, position in clicks_to_add:
            #history.append([item, new_grid_list.index(click)])
            #print(new_grid_list.index(item))
            result = np.where(new_grid == item)
            result_list = list(zip(result[0], result[1]))
            history.append([item, result_list])

    freqdist=update_freqdist(history, new_grid)
    return freqdist, history


In [19]:
oracle = UserOracle(maxdepth=3, associations=utilities.associations, activations= activations)

In [20]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state  # This could be a tuple of (grid, history)
        self.parent = parent
        self.children = {}
        self.num_visits = 0
        self.fully_expanded = False
        self.total_rewards = [0.0, 0.0, 0.0]

    def add_child(self, child_state, adaptation_key):
        child_node = Node(state=child_state)
        child_node.parent = self
        self.children[adaptation_key] = child_node
        return child_node

        

    def best_child(self, c_param=1.0):
        choices_weights = [
            (child.total_rewards[0] / child.num_visits) + c_param * math.sqrt((2 * math.log(self.num_visits) / child.num_visits))
            for child in self.children.values()
        ]
        return self.children[list(self.children.keys())[choices_weights.index(max(choices_weights))]]


### MCTS : association adaptations - Move

In [ ]:
class MCTS:
    def __init__(self, oracle, adaptations, initial_state, utilities, num_iterations=10, exploration_const=1.0/math.sqrt(2), objective='AVERAGE'):
        self.num_iterations = num_iterations  # No. of iterations to run
        self.exploration_const = exploration_const  # Original exploration constant: 1 / math.sqrt(2)

        self.oracle = oracle
        self.adaptations = adaptations
        self.utilities = utilities
        self.objective = objective
        self.initial_state = initial_state
        self.root = Node(state=self.initial_state)
        self.maxdepth=10
        self.weights = [0.25,0.5,0.25]

    def __str__(self):
        tree_str = str(self.root) + "\n"
        for child in self.root.children.values():
            tree_str += str(child) + "\n"
        return tree_str

    def execute_round(self):
        node = self.select_node(self.root)
        rewards = self.rollout(node.state)
        self.backpropagate(node, rewards)

    def search(self, initial_node=None):
        if initial_node:
            self.root = initial_node
            self.root.parent = None
        else:
            self.root = Node(state=self.initial_state)
        
        for _ in range(self.num_iterations):
            self.execute_round()

        #adaptation_probability = self.get_adaptation_probabilities(self.root, 0.0)

        best_child = self.get_best_child(self.root, 0.0)
        best_adaptation = self.get_adaptation(self.root, best_child)
        avg_rewards = [x / best_child.num_visits for x in best_child.total_rewards]

        return best_adaptation, best_child, avg_rewards

    def select_node(self, node):
        while not self.is_terminal(node.state):
            if node.fully_expanded:
                node = self.get_best_child(node, self.exploration_const)
            else:
                return self.expand(node)
        return node

    def expand(self, node):
        adaptations = self.adaptations.get_all_adaptations_within_bounding_box_mcts()
        random.shuffle(adaptations)
        for adaptation in adaptations:
            adaptation_key = (
                tuple(frozenset(adapt.items()) for adapt in adaptation['base_adaptation']),
                tuple(frozenset(adapt.items()) for adapt in adaptation['bbox_adaptation']) if adaptation['bbox_adaptation'][0] is not None else None
            )

            if adaptation_key not in node.children:
                new_state = self.apply_adaptation(node.state, adaptation)
                new_node = node.add_child(new_state, adaptation_key)
                if len(adaptations) == len(node.children) or self.is_terminal(new_node.state):
                    node.fully_expanded = True
                    print('fully expanded', bool(self.is_terminal(new_node.state)), len(adaptations), len(node.children), node.state.depth)
                return new_node
        raise Exception("Ouch! Should never reach here")

    def backpropagate(self, node, rewards):
        while node is not None:
            node.num_visits += 1
            node.total_rewards = [a + b for a, b in zip(node.total_rewards, rewards)]
            node = node.parent

    def get_best_child(self, node, exploration_const):
        best_value = float("-inf")
        best_node = None
        children = list(node.children.values())
        random.shuffle(children)
        for child in children:
            total_reward = self.compute_reward(child.total_rewards)
            node_value = total_reward / child.num_visits + exploration_const * math.sqrt(math.log(node.num_visits) / child.num_visits)
            if node_value > best_value:
                best_value = node_value
                best_node = child
        return best_node

    def compute_reward(self, total_rewards):
        if self.objective == "AVERAGE":
            total_reward = sum([a * b for a, b in zip(self.weights, total_rewards)])  # Take average reward
        elif self.objective == "OPTIMISTIC":
            total_reward = max(total_rewards)  # Take best reward
        elif self.objective == "CONSERVATIVE":
            total_reward = min(total_rewards) if min(total_rewards) >= 0 else min(total_rewards) * 2  # Take minimum; add penalty if negative
        return total_reward

    def get_adaptation(self, root, best_child):
        for adaptation, node in root.children.items():
            if node is best_child:
                return adaptation

    def get_adaptation_probabilities(self, node, exploration_const):
        if not node.children:
            return None
        probability = {a: 0.0 for a in self.adaptations.get_all_adaptations_within_bounding_box_mcts()}
        for adaptation, child in node.children.items():
            probability[adaptation] = child.num_visits / node.num_visits
        return probability

    def rollout(self, state):
            rewards = [0.0, 0.0, 0.0]
            while not self.is_terminal(state):
                try:
                    adaptation = random.choice(self.adaptations.get_all_adaptations_within_bounding_box_mcts())
                except IndexError:
                    raise Exception("Non-terminal state has no possible adaptations: " + str(state))
                state = self.apply_adaptation(state, adaptation)
                if state.exposed:
                    new_rewards = self.oracle.get_individual_rewards(state.ui_elements_grid_state, state.previous_seen_state.ui_elements_grid_state, state.freqdist)[0]
                    rewards = [a + b for a, b in zip(rewards, new_rewards)]
            return rewards
        


    def apply_adaptation(self, state, adaptation):
        grid, history = state.ui_elements_grid_state, state.indexed_history
        new_grid = self.adaptations.move_elements_mcts(adaptation)
        #print('\n new grid \n', new_grid, '\n for adaptaion \n', adaptation)
        freqdist, new_history = update_hist(history, new_grid)
        self.utilities.ui_elements_grid, self.utilities.indexed_history, self.utilities.freqdist = deepcopy(new_grid), deepcopy(new_history), deepcopy(freqdist)

        previous_seen_state = State(
            indexed_history=state.indexed_history.copy(),
            freqdist=state.freqdist.copy(),
            ui_elements_grid=state.ui_elements_grid_state.copy(),
            association_matrix=state.association_matrix.copy() if state.association_matrix is not None else None,
            depth=state.depth,
            previous_seen_state=None,
            exposed=state.exposed
        )
        #print('\n adaptation', adaptation, '\n grid \n', new_grid)
        new_state = deepcopy(state)
        new_state.previous_seen_state = previous_seen_state
        new_state.ui_elements_grid_state = deepcopy(new_grid)
        new_state.indexed_history = deepcopy(new_history)
        new_state.freqdist = deepcopy(freqdist)
        new_state.association_matrix = deepcopy(self.utilities.get_association_matrix())
        new_state.depth += 1

        self.adaptations.menustate.grid = deepcopy(new_grid)
        activations = self.utilities.get_activations(new_history, session_interval=50, duration_between_clicks=30)
        self.oracle.activations = deepcopy(activations)
        return new_state


    def is_terminal(self, state, depth=10):
        #print('depth output', node.state.depth)
        return state.depth >= depth  # Check depth or any other terminal condition

### MCTS: bbox adaptations - Swap

In [21]:
class MCTS:
    def __init__(self, oracle, adaptations, initial_state, utilities, num_iterations=200, exploration_const=1.0/math.sqrt(2), objective='AVERAGE'):
        self.num_iterations = num_iterations  # No. of iterations to run
        self.exploration_const = exploration_const  # Original exploration constant: 1 / math.sqrt(2)

        self.oracle = oracle
        self.adaptations = adaptations
        self.utilities = utilities
        self.objective = objective
        self.initial_state = initial_state
        self.root = Node(state=self.initial_state)
        self.maxdepth=10
        self.weights = [0.25,0.5,0.25]

    def __str__(self):
        tree_str = str(self.root) + "\n"
        for child in self.root.children.values():
            tree_str += str(child) + "\n"
        return tree_str

    def execute_round(self):
        node = self.select_node(self.root)
        rewards = self.rollout(node.state)
        self.backpropagate(node, rewards)

    def search(self, initial_node=None):
        if initial_node:
            self.root = initial_node
            self.root.parent = None
        else:
            self.root = Node(state=self.initial_state)
        
        for _ in range(self.num_iterations):
            self.execute_round()

        #adaptation_probability = self.get_adaptation_probabilities(self.root, 0.0)

        best_child = self.get_best_child(self.root, 0.0)
        best_adaptation = self.get_adaptation(self.root, best_child)
        avg_rewards = [x / best_child.num_visits for x in best_child.total_rewards]

        return best_adaptation, best_child, avg_rewards

    def select_node(self, node):
        while not self.is_terminal(node.state):
            if node.fully_expanded:
                node = self.get_best_child(node, self.exploration_const)
            else:
                return self.expand(node)
        return node

    def expand(self, node):
        adaptations = self.adaptations.get_all_swap_adaptations_within_bounding_box_mcts()
        random.shuffle(adaptations)
        for adaptation in adaptations:
            base_adaptation_key = tuple(frozenset(adapt.items()) for adapt in adaptation['base_adaptation'])
            bbox_adaptation_key = None
            if adaptation['bbox_adaptation'][0] is not None:
                bbox_adaptation_key = tuple(
                    frozenset(adapt.items()) for move_pair in adaptation['bbox_adaptation'] for adapt in move_pair
                )
            adaptation_key = (base_adaptation_key, bbox_adaptation_key)

            if adaptation_key not in node.children:
                new_state = self.apply_adaptation(node.state, adaptation)
                new_node = node.add_child(new_state, adaptation_key)
                if len(adaptations) == len(node.children) or self.is_terminal(new_node.state):
                    node.fully_expanded = True
                return new_node
        raise Exception("Ouch! Should never reach here")

    def backpropagate(self, node, rewards):
        while node is not None:
            node.num_visits += 1
            node.total_rewards = [a + b for a, b in zip(node.total_rewards, rewards)]
            node = node.parent

    def get_best_child(self, node, exploration_const):
        best_value = float("-inf")
        best_node = None
        children = list(node.children.values())
        random.shuffle(children)
        for child in children:
            total_reward = self.compute_reward(child.total_rewards)
            node_value = total_reward / child.num_visits + exploration_const * math.sqrt(math.log(node.num_visits) / child.num_visits)
            if node_value > best_value:
                best_value = node_value
                best_node = child
        return best_node

    def compute_reward(self, total_rewards):
        if self.objective == "AVERAGE":
            total_reward = sum([a * b for a, b in zip(self.weights, total_rewards)])  # Take average reward
        elif self.objective == "OPTIMISTIC":
            total_reward = max(total_rewards)  # Take best reward
        elif self.objective == "CONSERVATIVE":
            total_reward = min(total_rewards) if min(total_rewards) >= 0 else min(total_rewards) * 2  # Take minimum; add penalty if negative
        return total_reward

    def get_adaptation(self, root, best_child):
        for adaptation, node in root.children.items():
            if node is best_child:
                return adaptation

    def get_adaptation_probabilities(self, node, exploration_const):
        if not node.children:
            return None
        probability = {a: 0.0 for a in self.adaptations.get_all_swap_adaptations_within_bounding_box_mcts()}
        for adaptation, child in node.children.items():
            probability[adaptation] = child.num_visits / node.num_visits
        return probability

    def rollout(self, state):
            rewards = [0.0, 0.0, 0.0]
            while not self.is_terminal(state):
                try:
                    adaptation = random.choice(self.adaptations.get_all_swap_adaptations_within_bounding_box_mcts())
                except IndexError:
                    raise Exception("Non-terminal state has no possible adaptations: " + str(state))
                state = self.apply_adaptation(state, adaptation)
                if state.exposed:
                    new_rewards = self.oracle.get_individual_rewards(state.ui_elements_grid_state, state.previous_seen_state.ui_elements_grid_state, state.freqdist)[0]
                    rewards = [a + b for a, b in zip(rewards, new_rewards)]
            return rewards
        


    def apply_adaptation(self, state, adaptation):
        grid, history = state.ui_elements_grid_state, state.indexed_history
        new_grid = self.adaptations.apply_bbox_swap_adaptation_mcts(adaptation)
        #print('\n new grid \n', new_grid, '\n for adaptaion \n', adaptation)
        freqdist, new_history = update_hist(history, new_grid)
        self.utilities.ui_elements_grid, self.utilities.indexed_history, self.utilities.freqdist = deepcopy(new_grid), deepcopy(new_history), deepcopy(freqdist)

        previous_seen_state = State(
            indexed_history=state.indexed_history.copy(),
            freqdist=state.freqdist.copy(),
            ui_elements_grid=state.ui_elements_grid_state.copy(),
            association_matrix=state.association_matrix.copy() if state.association_matrix is not None else None,
            depth=state.depth,
            previous_seen_state=None,
            exposed=state.exposed
        )
        #print('\n adaptation', adaptation, '\n grid \n', new_grid)
        new_state = deepcopy(state)
        new_state.previous_seen_state = previous_seen_state
        new_state.ui_elements_grid_state = deepcopy(new_grid)
        new_state.indexed_history = deepcopy(new_history)
        new_state.freqdist = deepcopy(freqdist)
        new_state.association_matrix = deepcopy(self.utilities.get_association_matrix())
        new_state.depth += 1

        self.adaptations.menustate.grid = deepcopy(new_grid)
        activations = self.utilities.get_activations(new_history, session_interval=50, duration_between_clicks=30)
        self.oracle.activations = deepcopy(activations)
        return new_state


    def is_terminal(self, state, depth=10):
        #print('depth output', node.state.depth)
        return state.depth >= depth  # Check depth or any other terminal condition

In [22]:
# Initialize and run MCTS
mcts = MCTS(oracle, adaptations, root_state, utilities)

best_action = mcts.search()

print("Best action:", best_action[1].state.ui_elements_grid_state)

Best action: [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 ['B11' 'B18' 'B13' 0 0]
 ['B10' 'B14' 'B12' 0 0]
 ['B9' 'B20' 0 0 0]
 [0 'B17' 0 0 0]
 [0 'B16' 0 0 0]
 [0 'B19' 0 0 0]
 [0 'B15' 0 0 0]
 [0 0 0 0 0]
 [0 'B5' 0 0 0]
 [0 'B6' 0 0 0]
 [0 'B7' 0 0 0]
 [0 'B3' 0 0 0]
 [0 'B4' 0 0 0]
 [0 'B1' 0 0 0]
 [0 'B8' 0 0 0]
 [0 'B2' 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 

In [ ]:
best_action[2]

In [ ]:
print("Best action:", best_action[1].state.ui_elements_grid_state)

In [25]:
print(adaptations.menustate.grid)

[['B13' 'B15' 'B14' 0 'B17' 'B18' 'B16' 0 0 'B2' 'B1' 0 0 'B7' 0 'B6' 0
  'B5' 0 0 0 0 0 0 0 'B11' 'B10' 'B8' 'B9' 'B12' 0 'B4' 0 'B3' 0 0 0
  'B19' 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
#adaptations.menustate.update_bounding_boxes()
adaptations.get_all_adaptations_within_bounding_box()